We have a file named predictions.csv. This file contains in each row either a 0 or 1 for whether this sentence is to be added in the summary or not.
This csv has exaclty the same length with the test.csv file (for 300 rows articles) . So now that we have the values for each sentence we can create a txt file that has on the first column the id of the summary, and in the second column it has the 3 extracted sentences from the column article, which we have preprocessed.

In [ ]:
!pip install datasets contractions rouge rouge_score

In [ ]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
import datasets
from datasets import load_dataset, load_metric
from rouge_score import rouge_scorer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import string
nltk.download('wordnet')
import contractions
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm


In [ ]:
# Specify the dataset name and version
dataset_name = "cnn_dailymail"
dataset_version = "3.0.0"

# Load a specific version of the dataset from Hugging Face
dataset = datasets.load_dataset(dataset_name, dataset_version)

# Convert the dataset to a Pandas DataFrame
df = pd.DataFrame(dataset['test'])  # You can use 'train', 'test', or 'validation'
df.to_csv('test.csv')
#df =pd.read_csv('/content/output_file.csv')

In [ ]:
df1 =pd.read_csv('/content/test.csv')
df1=df.head(300)

Same preprocessing as for the training

In [ ]:
def apply_lemmatization(tokens):
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(word) for word in tokens]

def apply_stemming(tokens):
  porter = PorterStemmer()
  return [porter.stem(word) for word in tokens]

def clean_text(x):
  puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#',
              '*', '+', '\\', '•', '~', '@', '£',
              '·', '_', '{', '}', '©', '^', '®', '`','--', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â',
              '█', '½', 'à', '…',
              '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―',
              '¥', '▓', '—', '‹', '─',
              '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸',
              '¾', 'Ã', '⋅', '‘', '∞',
              '∙', '）', '↓', '、', '│', '（', '»','«', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø',
              '¹', '≤', '‡', '√', ]

  x = str(x)
  for punct in puncts:
    x = x.replace(punct, f' {punct} ')
  return x

def remove_non_ascii(tokens):
    return [word for word in tokens if re.match(r'^[^\x00-\x7F]+$', word) is None]

def clean_and_tokenize(article):
    # Remove text within parentheses and everything in it
    cleaned_article = re.sub(r'\([^)]*\)', '', article)

    # Split the text using '-- ' as the delimiter
    parts = re.split(r'-- ', cleaned_article, maxsplit=1)

    # Check if there was a match and reconstruct the text
    cleaned_article = parts[1] if len(parts) > 1 else cleaned_article

    # Remove 'E-mail to a friend' and anything that follows it
    cleaned_article = re.sub(r'E-mail to a friend.*', '', cleaned_article)

    # Expand word contractions
    expanded_article = contractions.fix(cleaned_article)
    # Add a period after the closing quotation mark if there is a space and a capital letter
    text_with_period = re.sub(r'(")([ ])([A-Z])', r'\1.\2\3', expanded_article)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text_with_period)
    filtered_article = [word for word in word_tokens if word.lower() not in stop_words and word.lower() != "'s"]

    # Replace country abbreviations with names
    filtered_article = ' '.join(filtered_article)

    sentences = sent_tokenize(filtered_article)
    # Lowercase the text
    sentences = [sentence.lower() for sentence in sentences]
    # Tokenize sentences
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
    # Remove punctuation using custom clean_text function
    tokenized_sentences_no_punct = [
        [word for word in word_tokenize(clean_text(sentence)) if word not in string.punctuation]
        for sentence in sentences]
    tokenized_sentences_no_ascii = [
    remove_non_ascii(sentence)
    for sentence in tokenized_sentences_no_punct]
    # Apply lemmatization
    tokenized_sentences_lemmatized = [
        apply_lemmatization(sentence)
        for sentence in tokenized_sentences_no_ascii
        if len(sentence) > 5
        ]

    # Remove empty lists
    tokenized_sentences_lemmatized = [sentence for sentence in tokenized_sentences_lemmatized if sentence]

    return tokenized_sentences_lemmatized

In [ ]:
df1['cleaned_article'] = df1['article'].apply(clean_and_tokenize)


In [ ]:
pd.set_option('display.max_colwidth', None)

# Display the DataFrame with the full content of the 'cleaned_article' column
print(df1[['cleaned_article']])


In [ ]:
df1['first_half_sentences'] = df1['cleaned_article'].apply(lambda sentences: sentences[:len(sentences)//2])


<ipython-input-7-da0504aae579>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['first_half_sentences'] = df1['cleaned_article'].apply(lambda sentences: sentences[:len(sentences)//2])


In [ ]:
# Explode the DataFrame and reset the index
df_test_exploded = df1.explode('first_half_sentences').reset_index(drop=True)

# Rename the 'articles' column to 'sentences'
df_test_exploded = df_test_exploded.rename(columns={'first_half_sentences': 'sentences'})

# Display the resulting DataFrame
print(df_test_exploded)

                                                article  \
0     (CNN)The Palestinian Authority officially beca...   
1     (CNN)The Palestinian Authority officially beca...   
2     (CNN)The Palestinian Authority officially beca...   
3     (CNN)The Palestinian Authority officially beca...   
4     (CNN)The Palestinian Authority officially beca...   
...                                                 ...   
2618  Surkhet, Nepal (CNN)Ten years ago, with her hi...   
2619  Surkhet, Nepal (CNN)Ten years ago, with her hi...   
2620  Surkhet, Nepal (CNN)Ten years ago, with her hi...   
2621  Surkhet, Nepal (CNN)Ten years ago, with her hi...   
2622  Surkhet, Nepal (CNN)Ten years ago, with her hi...   

                                             highlights  \
0     Membership gives the ICC jurisdiction over all...   
1     Membership gives the ICC jurisdiction over all...   
2     Membership gives the ICC jurisdiction over all...   
3     Membership gives the ICC jurisdiction over all...

In [ ]:
df_test_exploded.shape

Add the predictions csv

In [ ]:
preds=pd.read_csv('/content/drive/MyDrive/NLU/Task_2/dataset/predictions.csv')
preds

Add the dataframe predictionns in the df_test, since they have exactly same shape

In [ ]:
# Concatenate the DataFrames along columns
combi = pd.concat([df_test_exploded, preds], axis=1)
combi

In [ ]:
# Extract unique 'id' values in the order they appear
unique_ids = combi['id'].unique()

# Initialize a variable to store all summaries
all_summaries = []

# Iterate through unique 'id' values in the original order
for article_id in unique_ids:
    # Filter the DataFrame for the specific 'id'
    group = combi[combi['id'] == article_id]
    extracted_sentences = group.loc[group['y_pred'] == 1, 'sentences'].tolist()

    # Ensure each element in 'extracted_sentences' is a list
    extracted_sentences = [sentence if isinstance(sentence, list) else [] for sentence in extracted_sentences]

    # Convert numbers to strings and add full stop as a separator
    extracted_sentences = [[' '.join(map(str, sentence)) + '.' for sentence in extracted_sentences]]

    # Append the summary to the list
    summary = f'({article_id}) {" ".join(" ".join(sentence) for sentence in extracted_sentences)}'
    all_summaries.append(summary)

# Create a text file and write all summaries
with open('all_summaries.txt', 'w') as file:
    file.write('\n'.join(all_summaries))

In [ ]:
# Function to calculate Rouge-2 scores
def calculate_rouge_scores(highlight, summary, scorer):
    scores = scorer.score(highlight, summary)
    return {'precision': round(scores['rouge2'].precision, 3),
            'recall': round(scores['rouge2'].recall, 3),
            'fmeasure': round(scores['rouge2'].fmeasure, 3)}

# Extract unique 'id' values in the order they appear
unique_ids = combi['id'].unique()

# Initialize a variable to store article data and Rouge-2 scores
article_data = []

# Create a RougeScorer instance
scorer = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=False)
# Iterate through unique 'id' values in the original order
for article_id in unique_ids:
    # Filter the DataFrame for the specific 'id'
    group = combi[combi['id'] == article_id]
    extracted_sentences = group.loc[group['y_pred'] == 1, 'sentences'].tolist()
    highlight = group['highlights'].iloc[0]  # Assume the highlight is the same for all sentences

    # Convert each element to string and add full stop as a separator
    extracted_sentences = [' '.join(map(str, map(str, element))) + '.' if isinstance(element, list) else '' for element in extracted_sentences]

    # Calculate Rouge-2 scores
    rouge_scores = calculate_rouge_scores(highlight, ' '.join(extracted_sentences), scorer)

    # Append the article data to the list
    article_data.append({
        'id': article_id,
        'highlight': highlight,
        'summary': ' '.join(extracted_sentences),
        'rouge2_scores': rouge_scores,
    })

# Create a DataFrame from the article data
result_df = pd.DataFrame(article_data)

# Display the resulting DataFrame
print(result_df)


                                           id  \
0    f001ec5c4704938247d27a44948eebb37ae98d01   
1    230c522854991d053fe98a718b1defa077a8efef   
2    4495ba8f3a340d97a9df1476f8a35502bcce1f69   
3    a38e72fed88684ec8d60dd5856282e999dc8c0ca   
4    c27cf1b136cc270023de959e7ab24638021bc43f   
..                                        ...   
295  f626c9207fe036e2ed8e30df692ba94b7999cad7   
296  1203b98b4a1739d0f88789ea59f4cceffde40f2d   
297  1ea2893555b80472c3b690b00dc9223c8f71e829   
298  5c4eab115efd57b8283c0eb0b49b94ba585b8062   
299  275f70b420d88b7c1e6140d5b4586a2ea36a8015   

                                             highlight  \
0    Membership gives the ICC jurisdiction over all...   
1    Theia, a bully breed mix, was apparently hit b...   
2    Mohammad Javad Zarif has spent more time with ...   
3    17 Americans were exposed to the Ebola virus w...   
4    Student is no longer on Duke University campus...   
..                                                 ...   
295  

#Issues: the summary column has no stopwords and pnctuation, whereas the highlights has everything, has no preprocessing.

Maybe we should think of smarter way to write the summaries. So tht the original sentences are added in the summary and can compared to the original highlights

#Rouge_2 metrics

In [ ]:
# Assuming you already have the 'result_df' DataFrame with 'rouge2_scores' column
# Convert 'rouge2_scores' column to a DataFrame with three separate columns
rouge2_df = pd.DataFrame(result_df['rouge2_scores'].tolist())

# Calculate mean and std for precision, recall, and fmeasure
mean_precision = rouge2_df['precision'].mean()
std_precision = rouge2_df['precision'].std()

mean_recall = rouge2_df['recall'].mean()
std_recall = rouge2_df['recall'].std()

mean_fmeasure = rouge2_df['fmeasure'].mean()
std_fmeasure = rouge2_df['fmeasure'].std()

# Display the results
print(f"Mean Precision: {mean_precision:.3f}, Std Precision: {std_precision:.3f}")
print(f"Mean Recall: {mean_recall:.3f}, Std Recall: {std_recall:.3f}")
print(f"Mean F-measure: {mean_fmeasure:.3f}, Std F-measure: {std_fmeasure:.3f}")


Mean Precision: 0.036, Std Precision: 0.045
Mean Recall: 0.048, Std Recall: 0.063
Mean F-measure: 0.040, Std F-measure: 0.050
